In [ ]:
import sys
import os
from pathlib import Path
new_version_path = Path(os.getenv('VNPY2.0.8'))
sys.path.insert(0, str(new_version_path))

import vnpy
vnpy

In [ ]:
import pandas as pd
import multiprocessing

from itertools import product
from pathlib import Path
from datetime import datetime
from vnpy.app.cta_strategy.backtesting import OptimizationSetting
from utility import continuous_backtest, segment_backtest, merge_result, get_output_path
          
def multi_run(gen_setting: dict, settings_list: list):    
    pool = multiprocessing.Pool(multiprocessing.cpu_count())
    results = []
    for setting_kwargs in settings_list:
        print(f"begin{setting_kwargs}")
        gen_setting.update(setting_kwargs)
        print(gen_setting)
        result = pool.apply_async(merge_result, kwds=gen_setting)
        results.append(result)
    pool.close()
    pool.join()
    
    result_values = [result.get() for result in results]
#     result_values = [result for result in result_values if result]
    return result_values

def sync_run(gen_setting: dict, settings_list: list):
    results = []
    for setting_kwargs in settings_list:
        gen_setting.update(setting_kwargs)
        res = merge_result(**gen_setting)
        if res:
            results.append(res)
    print("All finished!")
    return results
    

### 多组回测

In [ ]:
commodities = [
    "cu", "al", "zn", "pb", "ag", "rb", "ru", "bu", "hc", "au", "ni",
    "jd", "p", "j", "m", "y", "jm", "a", "i", "l", "pp", "cs", "c", "v",
    "SF", "SM", "SR", "CF", "TA", "MA", "RM", "OI", "FG", "ZC",
    "IF", "IH", "IC",
    'sc'
]

# commodities = ["rb"]

boll_gen = OptimizationSetting()
boll_gen.add_parameter("boll_window", 20, 40, 10)
boll_gen.add_parameter("sl_multiplier", 2, 4, 1)
boll_settings = boll_gen.generate_setting()

turtle_gen = OptimizationSetting()
turtle_gen.add_parameter("entry_window", 25, 55, 15)
turtle_gen.add_parameter("exit_window", 10, 20, 5)
turtle_settings = turtle_gen.generate_setting()

boll_params_values = list(product(commodities, boll_settings, ['boll']))
turtle_params_values = list(product(commodities, turtle_settings, ['turtle']))

params_keys = ['commodity','strategy_params', 'strategy_name']
boll_kwargs_list = [dict(zip(params_keys, value_tuple)) for value_tuple in boll_params_values]
turtle_kwargs_list = [dict(zip(params_keys, value_tuple)) for value_tuple in turtle_params_values]
all_kwargs_list = boll_kwargs_list + turtle_kwargs_list

gen_settings= {
    'interval': '1h',
    'capital': 100000,
    'start': datetime(2017, 1, 1),
    'end': datetime(2019, 12, 1)
}

# res_list = multi_run(gen_settings, all_kwargs_list)
res_list = sync_run(gen_settings, all_kwargs_list)
df = pd.DataFrame(res_list)
df.to_csv(get_output_path('batch_result.csv'))

### 单例回测

In [ ]:
commodity = 'sc'
data_mode = 'main'
interval = '1h'
strategy_name = 'turtle'
strategy_params = {'entry_window': 25, 'exit_window': 10, 'fixed_size': 1}
capital = 100000
start = datetime(2017, 1, 1)
end = datetime(2019, 12, 1)

# continuous_backtest(commodity, data_mode, interval, strategy_name, strategy_params, capital, start, end)
# segment_backtest(commodity, interval, strategy_name, strategy_params, capital, start, end)
# print(f'{commodity}.{strategy_name}.{strategy_params} test over')

single_res = merge_result(commodity, interval, strategy_name, strategy_params, capital, start, end)